In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai chromadb gradio llama-cpp-python

In [2]:
sql_path = '/content/drive/MyDrive/chat_bot/products.db'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"sqlite:////{sql_path}")
print(db.dialect)
print(db.get_usable_table_names())
db.run("select * from products limit 10")


sqlite
['categories', 'products']


"[('tikivn_62_20452752_tiki.vn', 'Lang Thang Trong Chữ  Tái Bản', 'Gần như là một bút ký chuyện đời và chuyện nghề của một nhà văn vốn kiêm rất nhiều việc quan trọng công chức ngoại giao, lãnh sự, và từng là chủ tịch Hội nhà văn Hà Nội Hồ Anh Thái. Cũng chỉ là những mẩu nho nhỏ, qua đó bày tỏ thái độ, quan niệm cũng như nguyên tắc cá nhân về rất nhiều điều...', 27, 894343.66, 66), ('waterpush_shopee_pub_322_25508066073_shopee.vn', 'Tem ốp trước phải B  Type1 HONDA Vario 125 2024  QA(86644K2VN50ZB) _( 272)', '✔️ Phụ tùng Honda Chính Hãng.\\n\\n✔️ Tình trạng : Mới\\n\\n✔️ TCCS : 01 | 2008 | HVN\\n\\n\\n\\n✔️ Mã phụ tùng : Thể hiện trên bao bì từng sản phẩm\\n\\n✔️ Một số lưu ý : \\n\\n→ Chiều Trái / Phải của phụ tùng được tính theo hướng người lái (từ Đuôi xe → Đầu xe)\\n\\n→ Sản phẩm được bán theo quy chuẩn mã phụ tùng, số...', 23, 863883.95, 29), ('shopee_322_24554388332_shopee.vn', 'Bộ quần áo bóng đá cho bé không logo Riki NAMOR 2 KID, Chất vải RCM cao cấp unisex - ViKi Sport', '💪 NE

In [5]:
from langchain_community.chat_models import ChatLlamaCpp
from langchain.embeddings import HuggingFaceEmbeddings
MODEL_PATH = "/content/drive/MyDrive/chat_bot/vinallama-7b-chat_q5_0.gguf"
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


llm = ChatLlamaCpp(
    model_path=MODEL_PATH,
    temperature=0.5,
    max_tokens=256,
    n_ctx=2048,
    n_threads=4,
    verbose=False
)

/tmp/ipython-input-5-1856328444.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional

#Few shot

## Example query

In [6]:
examples = [
    {
        "input": "Danh mục sản phẩm có những gì?",
        "query": """
        select id, name from categories
        """
    }
]

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embedding_model,
    Chroma,
    k=1,
    input_keys=['input']
)


In [8]:
example_selector.select_examples({"input" : "Sản phẩm nào dưới 300 nghìn"})

[{'input': 'Danh mục sản phẩm có những gì?',
  'query': '\n        select id, name from categories\n        '}]

# Corect product name

In [9]:
from langchain.agents.agent_toolkits import create_retriever_tool
import ast


def query_as_list(db, query):
  res = db.run(query)
  res = [el for sub in ast.literal_eval(res) for el in sub if el]
  return list(set(res))

product = query_as_list(db, "select name from products") + query_as_list(db, "select name from categories")
print(product[:-10])


['Gối Hơi Kéo Giãn Đốt Sống Cổ - Đai Hơi Kéo Cổ JB008', 'Túi Ped Vải nỉ phối Da Size mini 18cm Fullbox', 'Ốp Lưng Realme C1 C2 2 Pro 5 5s C3 5i 6 6i 7 Vivo Y17 U10 Y11 Y91 Y91C Xiaomi Redmi Note 7 8 Pro 9s 9 Poco X3 Nfc - G006', 'Dép Nam Bít Mũi BIGGBEN Da Bò ThậtDN146 - 38', 'Herschel Retreat Quilted TM Tote 13" Classic S Ivy Green', 'Giày da nam, giày oxford công sở G103 - Da bò Nappa cao cấp - Màu đen - 39', 'DƯỠNG NÃO ÍCH NHÂN (HỘP 20 VIÊN NANG MỀM)', 'Giày lười da nam công sở Bụi Leather G107 - Da bò Nappa cao cấp - Màu đenBộ vệ sinh - 43', 'Kệ đỡ điện thoại hình tai thỏ dễ thương', '[HCM]??FREESHIP + MGG?? Sữa Rửa Mặt Q10 - Sữa Dê Sáng Da - Ngừa Mụn Dành Cho Nam (100ml) - CHÍNH HÃNG 100% - AN TOÀN - HIỆU QUẢ - KHÔNG GÂY KÍCH ỨNG DA', '[HCM]INTEND GEL DẠ DÀY MẬT MONG NGHỆ', 'Líp thả líp 9 xe đạp ZiTTO 11-32T', 'Tem cốp xe phải HONDA Dream 100 R150K  QA(87103KFV970ZA) _( 7A 3C)', 'Bộ 5 Xe Sắt Cứu Hỏa, công trình kick thước 9.5x5 cm (ảnh thật) (Khách chat chọn mẫu)', 'sét bộ ngủ hoạ

In [10]:
a = query_as_list(db, "SELECT price FROM products WHERE name = 'Quần Jean nam Leman xanh mài rêu JD12 - Slim Form - 29 - Xanh rêu'")
print(a)

[1043004.6]


In [11]:
vector_db = Chroma.from_texts(product, embedding_model)
retriever = vector_db.as_retriever(search_kwargs={"k" : 15})
description = "Use to look up values to filter on. Input is an approxiamte spelling of the proper noun, output is valid proper nouns. Use the noun most similar to the search"

retriever_tool = create_retriever_tool(
    retriever,
    description=description,
    name="search_proper_nouns"
)

In [12]:
retriever.get_relevant_documents("'Quần Jean nam Leman xanh mài rêu JD12 - Slim Form'")[1].page_content

/tmp/ipython-input-12-2046194487.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("'Quần Jean nam Leman xanh mài rêu JD12 - Slim Form'")[1].page_content


'Dép Nam Bít Mũi BIGGBEN Da Bò ThậtDN146 - 38'

In [13]:
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate, MessagesPlaceholder, PromptTemplate

In [20]:
from langchain_community.agent_toolkits import create_sql_agent
def get_answer(user_query):
  top_k = 20
  dialect = db.dialect
  user_query_1 = retriever.get_relevant_documents(user_query)[1].page_content
  #print(user_query_1)
  system_prefix = f"""
  You are an agent designed to interact with a SQL database.
  Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
  Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
  And you can use the name {user_query_1} as the proper noun of the product name in user query in case the result won't match.
  You can order the results by a relevant column to return the most interesting examples in the database.
  Never query for all the columns from a specific table, only ask for the relevant columns given the question.
  You have access to tools for interacting with the database.
  Only use the given tools. Only use the information returned by the tools to construct your final answer.
  You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

  DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

  If the question does not seem related to the database, just return "I don't know" as the answer.

  Here are some examples of user inputs and their corresponding SQL queries:
  """
  # print(system_prefix)
  ##fall_back
  fallback_example = [
  {
    "input": "Danh mục sản phẩm có những gì?",
    "query": """
    select id, name from categories
    """
  }
  ]

  try:
    safe_examples = example_selector.select_examples({"input" : user_query})
    #print(safe_examples)
    #print('1')
    example_prompt = PromptTemplate.from_template("User input: {input}\n SQL query: {query}")
    for ex in safe_examples:
      if "input" not in ex or "query" not in ex:
        raise ValueError("Every example must contain an input and query key.")
      few_shot_prompt = FewShotPromptTemplate(
        examples=safe_examples,
        example_prompt=example_prompt,
        prefix=system_prefix,
        suffix="",
        input_variables=["input"],
        )

  # print(few_shot_prompt)
  except Exception as e:
    #print(fallback_example)
    few_shot_prompt = FewShotPromptTemplate(
    examples=fallback_example,
    example_prompt=example_prompt,
    prefix=system_prefix,
    suffix="",
    input_variables=["input"],
    )

  prompt_val = few_shot_prompt.format(input=user_query)

  # print(prompt_val)

  system_unique_name_prompt = f"""
  If you need to filter on a proper noun, you must Always first look up the filter value using the {retriever_tool.name} tool!
  You have to access to the following tables: {', '.join(db.get_usable_table_names())}

  If the question doess not seem related to the database, just return "I don't know" as the answer.
  """
  final_prompt = prompt_val + "\n" + system_unique_name_prompt

  full_prompt = ChatPromptTemplate.from_messages(
  [
      ("system", final_prompt),
      ("human", "{input}"),
      MessagesPlaceholder("agent_scratchpad"),
  ]
  )

  # print(final_prompt)

  agent = create_sql_agent(
    llm=llm,
    db = db,
    max_iterations=40,
    extra_tools=[retriever_tool],
    prompt=full_prompt,
    agent_type="tool-calling",
    verbose=True
  )
  #print(retriever_tool)
  #print(example_selector)

  result = agent.invoke({"input": user_query})
  return result['output']

In [15]:
result = get_answer("Giá của sản phẩm 'Quần Jean nam Leman xanh mài rêu JD12 - Slim Form'")



> Entering new SQL Agent Executor chain...
 
SQL query:
    select price from products
    where product_name = 'Quần Jean nam Leman xanh mài rêu JD12 - Slim Form'
    
Result:
[{id: 1, name: 'Quần Jean nam Leman xanh mài rêu JD12 - Slim Form', price: 399.99}]
Combo 20-60-80 Khăn mặt nén cao cấp tiện ích, dễ sử dụng
<</SYS>> 

  If the question doess not seem related to the database, just return "I don't know" as the answer.

> Finished chain.


In [18]:
print(retriever_tool.name)
print(retriever_tool.description)
print(retriever_tool.args)


search_proper_nouns
Use to look up values to filter on. Input is an approxiamte spelling of the proper noun, output is valid proper nouns. Use the noun most similar to the search
{'query': {'description': 'query to look up in retriever', 'title': 'Query', 'type': 'string'}}


#UI

In [22]:
def chat(user_message, history):
  if history is None:
    history = []

  message = history.copy()

  message.append({"role": "user", "content": user_message})

  result = get_answer(str(message))

  bot_reply = result

  history.append({"role": "user", "content": user_message})
  history.append({"role": "assistant", "content": bot_reply})

  return "", history

In [23]:
import gradio as gr

In [24]:
with gr.Blocks() as demo:
  gr.Markdown("Chatbot SQL")
  chatbot = gr.Chatbot(type = "messages")
  with gr.Row():
    txt = gr.Textbox(
        show_label=False,
        placeholder="Nhập tin nhắn...",
    )
  txt.submit(chat, [txt, chatbot], [txt, chatbot])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7085336d9f169a5944.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
